In [1]:

# підключення потрібних модулей
import numpy as np
import pandas as pd

# import matplotlib.pyplot as plt
# import seaborn as sns

# параметри виведення
pd.set_option("display.max_columns", 500)  # кількість колонок
pd.set_option("display.max_rows", 1000)  # кількість рядків
pd.set_option("display.max_colwidth", 300)  # ширина колонок
pd.set_option("display.precision", 5)  # кількість знаків після коми

# вимикаємо зайві попередження
import warnings

warnings.filterwarnings("ignore")

# друк всіх результатів в одній комірці а не тільки останнього
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:

# завантаження ML-модулей
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer

# from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score

# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import balanced_accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier

# from sklearn.decomposition import PCA

from sklearn.base import BaseEstimator, TransformerMixin

In [3]:

# завантаження та первинний огляд дата-сету
data = pd.read_csv(
    "/home/negan/GoITProjects/MLT_FP/final_proj_data.csv",
)
print(data.shape)
print(data.info())
print(data.describe())
print(data.nunique())

(10000, 231)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 231 entries, Var1 to y
dtypes: float64(191), int64(2), object(38)
memory usage: 17.6+ MB
None
            Var1   Var2         Var3       Var4         Var5         Var6  \
count  133.00000  266.0    266.00000  280.00000  2.41000e+02   8980.00000   
mean    14.97744    0.0    341.05263    0.09643  2.33810e+05   1340.91626   
std     66.45601    0.0   2810.60697    0.92824  5.53231e+05   2380.51676   
min      0.00000    0.0      0.00000    0.00000  0.00000e+00      0.00000   
25%      0.00000    0.0      0.00000    0.00000  0.00000e+00    523.25000   
50%      0.00000    0.0      0.00000    0.00000  0.00000e+00    861.00000   
75%     16.00000    0.0      0.00000    0.00000  1.17235e+05   1428.00000   
max    680.00000    0.0  42588.00000    9.00000  3.02400e+06  76195.00000   

             Var7  Var8        Var9        Var10      Var11       Var12  \
count  8995.00000   0.0   133.00000  2.4

In [4]:


# клас об'єктів для видалення пустих значень
class NanDropTransformer(BaseEstimator, TransformerMixin):

    def __init__(
        self,
        col_trh=1.0,
        row_trh=1.0,
    ):
        self.col_trh = col_trh  # трешхолд для частки пустих значень в колонках
        self.row_trh = row_trh  # трешхолд для частки пустих значень в рядку
        self.drop_columns = None
        self.leave_row = None

    def fit(self, X, y=None):
        column_nan_ratio = X.isna().mean()
        nan_columns = column_nan_ratio[column_nan_ratio.ge(self.col_trh)].index
        non_info_columns = X.columns[X.nunique().le(1)]
        self.drop_columns = set(nan_columns) | set(non_info_columns)
        return self

    def transform(
        self,
        X,
        y=None,
    ):
        X = X.copy()
        # drop columns
        X.drop(self.drop_columns, axis=1, inplace=True)
        # drop row
        if y is not None:
            self.leave_row = X.isna().mean(axis=1).lt(self.row_trh)
            X = X[self.leave_row]
            y = y.copy()
            y = y[self.leave_row]
        return X, y


In [5]:


# клас об'єктів-трансформерів для логарифмування числових значень
# не буде використовуватися, бо PowerTransformer показав кращі результати
class LogTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, passthrow=[]):
        self.passthrow = passthrow # колонки, які виключаємо з логарифмування
        self.log_columns = []

    def fit(self, X, y=None):
        self.log_columns = [i for i in X.columns if i not in self.passthrow]
        return self

    def transform(self, X, y=None):
        X = X.copy()
        for col in self.log_columns:
            new_col = "Log" + col
            X[new_col] = X[col].apply(np.log1p)
        X.drop(self.log_columns, axis=1, inplace=True)
        return X

    # заглушка для сумісності з цим методом в пайплайні
    def set_output(self, transform=None):
        return self


In [6]:


# клас об'єктів-трансформерів для видалення колонок більшою кардинальністю, ніж задано
class CardinalTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, card_trh=None):
        self.card_trh = card_trh
        self.high_cardinal_columns = []

    def fit(self, X, y=None):
        self.high_cardinal_columns = X.columns[X.nunique().gt(self.card_trh)]
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X.drop(self.high_cardinal_columns, axis=1, inplace=True)
        print(len(self.high_cardinal_columns))
        return X

    # заглушка для сумісності з цим методом в пайплайні
    def set_output(self, transform=None):
        return self


In [7]:

# сам пайп-лайн

# оптимальні параметри (знайдені методом підбору)
col_nan_trh = 0.65
row_nan_trh = 0.10
cardinal_trh = 1500


num_transformer = Pipeline(
    steps=[
        # ('logarithmization', LogTransformer(['Var57', 'Var143', 'Var173'])),
        ("imputer", SimpleImputer(missing_values=pd.NA, strategy="mean")),
        (
            "powertransformer",
            PowerTransformer(),
        ),  # метод також нормує дані, тому StandardScaler не потрібен
        # ('scaler', StandardScaler()),
    ]
)

cat_transformer = Pipeline(
    steps=[
        ("decardinalization", CardinalTransformer(card_trh=cardinal_trh)),
        ("imputer", SimpleImputer(missing_values=pd.NA, strategy="most_frequent")),
        ("encoder", TargetEncoder(target_type="binary", random_state=42)),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer, make_column_selector(dtype_include=object)),
        ("num", num_transformer, make_column_selector(dtype_include=np.number)),
    ],
    n_jobs=-1,
    verbose_feature_names_out=False,
).set_output(transform="pandas")

model_pipeline = Pipeline(
    steps=[
        ("pre_processor", preprocessor),
        ("SMOTE", SMOTE(random_state=42, k_neighbors=24)),
        # ('pca', PCA(random_state=42, n_components=10)), # тільки погіршував результат
        (
            "clf_estimator",
            GradientBoostingClassifier(
                subsample=0.75,
                n_estimators=300,
                # learning_rate=0.3,
                max_depth=6,
                max_features="sqrt",
                random_state=42,
            ),
        ),
    ]
)

In [ ]:

# пошук найкращих параметрів

# X = data.copy()
# y = X.pop('y')

# nan_dropper = NanDropTransformer(col_trh=col_nan_trh, row_trh=row_nan_trh)
# nan_dropper.fit(X, y)
# X_full, y_full = nan_dropper.transform(X, y)

# # X_full.select_dtypes(include='object').nunique().sort_values(ascending=False)

# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# from sklearn.model_selection import GridSearchCV

# parameters = {
#     'clf_estimator__learning_rate': (0.1, 0.3),
#     'clf_estimator__subsample': (0.75, 0.85),
#     'clf_estimator__max_features': ('sqrt', 'log2'),
#     'clf_estimator__max_depth': (3, 4, 5, 6, 7, 8, 9, 10),
#     'clf_estimator__n_estimators': (100, 150, 200, 250, 300, 350, 400, 450, 500),
#     }

# search = (GridSearchCV(
#     estimator=model_pipeline,
#     param_grid=parameters,
#     scoring='balanced_accuracy',
#     cv=kf,
#     refit=False)
#     .fit(X, y))


# parameters_best = search.best_params_
# print(parameters_best)
# # pipe_upd = pipe_upd.set_params(**parameters_best)

# # model_upd = pipe_upd.fit(X, y)

# # cv_results = cross_val_score(
# #     estimator=model_pipeline,
# #     X=X_full,
# #     y=y_full,
# #     scoring='balanced_accuracy',
# #     cv=kf,
# #     verbose=0)

# # print(cv_results)

# # bal_acc = np.abs(cv_results).mean()

# # print(f"Pipe's balanced accuracy on CV: {bal_acc: 0.2%}")

In [8]:

# тестування моделі за допомогою крос-валідації

X = data.copy()
y = X.pop("y")

# видаляємо відсутні значення по параметрам
nan_dropper = NanDropTransformer(col_trh=col_nan_trh, row_trh=row_nan_trh)
nan_dropper.fit(X, y)
X_full, y_full = nan_dropper.transform(X, y)

# готуємо збалансовані фолди для крос-валідації
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_results = cross_val_score(
    estimator=model_pipeline,
    X=X_full,
    y=y_full,
    scoring="balanced_accuracy",
    cv=kf,
    verbose=0,
)

# оглядаємо результати по фолдах
print(cv_results)

# розраховуємо середнє значення
bal_acc = np.abs(cv_results).mean()

print(f"Pipe's balanced accuracy on CV: {bal_acc: 0.2%}")

# the best result = 87.40% (300/6/0.65/0.10/1500)

NanDropTransformer(col_trh=0.65, row_trh=0.1)

7
7
7
7
7
7
7
7
7
7
[0.87590099 0.87743168 0.87284485 0.88226009 0.89810437]
Pipe's balanced accuracy on CV:  88.13%


In [ ]:

# пошук найкращої стратегії видалення пустих значень

# result_string = ''

# for nan_trh in [0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]:
#     for cardinal_trh in [100, 200, 300, 500, 1000, 2000]:

#         X = data.copy()
#         y = X.pop('y')

#         nan_dropper = NanDropTransformer(col_trh=nan_trh, row_trh=nan_trh)
#         nan_dropper.fit(X, y)
#         X_full, y_full = nan_dropper.transform(X, y)

#         cv_results = cross_val_score(
#             estimator=model_pipeline,
#             X=X_full,
#             y=y_full,
#             scoring='balanced_accuracy',
#             cv=5,
#             verbose=0)

#         # print(cv_results)

#         bal_acc = np.abs(cv_results).mean()

#         result_string += f'nan_trh={nan_trh: 0.2f}, cardinal_trh={cardinal_trh: 0.2f}, bal_acc={bal_acc: 0.2%}\n'

# print(result_string)

# the best result = 77.26% before submitting, 80.39% after start submitting


In [9]:

# тренування моделі на всьому тренувальному дата-сеті
model = model_pipeline.fit(X_full, y_full)

# завантаження валідаційного дата-сету
valid = pd.read_csv("/home/negan/GoITProjects/MLT_FP/final_proj_test.csv")

# розрахунок передбачення
X_valid, _ = nan_dropper.transform(valid)
y_valid_predict = model.predict(X_valid)

8
8


In [10]:

# зберіганеня предікту в файл для сабмішену в кагл
output = pd.DataFrame(y_valid_predict, columns=["y"])

output.to_csv(
    "/home/negan/GoITProjects/MLT_FP/v6.17(GB_6-300_010_trh_65_10_ctrh_1500_power_mean_smote24_nopca).csv",
    index_label="index",
)